# Lab 7

Name: 

Date: 

**As you work through the code below, respond to all in-line comments and questions.**

This example will make use of the Newgroup data.  20 Newsgroup data can be found at:
http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html.  Some of the blocks of code are from Deep Learning with Python, 1st Edition, by Francois Chollet.

In [ ]:
!mkdir -p /opt/data/news20
!wget http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.tar.gz -O /opt/data/news20.tar.gz

!tar xvf /opt/data/news20.tar.gz -C /opt/data/news20 > /dev/null

In [ ]:
import os, sys

BASE_DIR = os.path.join(*['/','opt','data',])
TEXT_DATA_DIR = os.path.join(BASE_DIR, 'news20','20_newsgroup')

texts = []  # list of text samples
labels_index = {}  # dictionary mapping label name to numeric id
labels = []  # list of label ids
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        label_id = len(labels_index)
        labels_index[name] = label_id
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                args = {} if sys.version_info < (3,) else {'encoding': 'latin-1'}
                with open(fpath, **args) as f:
                    t = f.read()
                    i = t.find('\n\n')  # skip header
                    if 0 < i:
                        t = t[i:]
                    texts.append(t)
                labels.append(label_id)

print('Found %s texts.' % len(texts))
print(labels[1000], texts[1000])

In [3]:
# Listing 6.9 from DLP

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000

import numpy as np
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)  # The maximum number of words that can be converted to sequences
tokenizer.fit_on_texts(texts) # Need to examine the text to determine what the most frequent words are
                                #  and retain these.

sequences = tokenizer.texts_to_sequences(texts) # tokenizes and converts most common tokens to sequences

word_index = tokenizer.word_index
data = tokenizer.texts_to_matrix(texts, mode='binary') # Vector indicating presense of words in text


In [ ]:
# Listing 6.9 from DLP

from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences

VALIDATION_SPLIT = 0.2

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
targets = to_categorical(np.asarray(labels))

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
targets = targets[indices]

print('Shape of data tensor:', data.shape)
print('Shape of targets tensor:', targets.shape)

N = int(VALIDATION_SPLIT * data.shape[0])
train_X = data[:-N]
train_Y = targets[:-N]
val_X = data[-N:]
val_Y = targets[-N:]


Now train a simple classifier using the hot encoded input.

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding, Dropout



If we wanted use word embeddings instead of a hot-encoded vector, we could start by learning some word embeddings.

In [ ]:
EMBEDDING_DIM = 100




Let's used some pre-trained word embeddings such as [GloVe](https://nlp.stanford.edu/projects/glove/).  

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip -O /opt/data/glove.6B.zip
!unzip /opt/data/glove.6B.zip -d /opt/data/


In [11]:
# Listing 6.10 from DLP

import os

GLOVE_PATH = os.path.join(BASE_DIR, 'glove.6B.100d.txt')

embeddings_index = {}
with open(GLOVE_PATH) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))


Found 400000 word vectors.


In [13]:
# Listing 6.11 from DLP

embedding_matrix = np.zeros((MAX_NUM_WORDS, EMBEDDING_DIM))
for word, i in word_index.items():
  if i < MAX_NUM_WORDS:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
      embedding_matrix[i] = embedding_vector



Try training and evaluating a model that used existing word embeddings.